In [1]:
import numpy as np
import matplotlib.pyplot as plt
import neuron
from neuron import h, rxd,nrn
from neuron.units import ms, mV  #explicitly define our units
import math as m
from scipy.signal import find_peaks
import pandas as pd

In [2]:
#Include new mechanisms for additional channels 

h.nrn_load_dll('C:\\nrn\\Mod Files\\nrnmech.dll')

#h.load_file("nrnmech.dll")
h.load_file('stdrun.hoc') #loads in Neurons running domain

1.0

In [3]:
class Cortical_Neuron():
    # Class to create cortical neuron with relevant mechanisms and morphology

    def __init__(self, node_num=21, M_diam= 1.4, N_diam = 1.2):  # allow myelin and node diameter to be set each time
        # Create topology
        # self.connect = connect
        # self.connection_node = connection_node
        
        
        self.node_num = node_num
        self.nodes = []
        self.myelins = []
        for i in range(node_num):
            self.nodes.append(h.Section(name=f"Node_{i}"))  # Create sections with names corresponding to nodes Node_0 Node_1
            self.myelins.append(h.Section(name=f"Myelins_{i}"))

        self.ais = h.Section(name='ais')
        self.collateral = h.Section(name='collateral')

        # Connect nodes and myelin into one long axon
        for i in range(node_num-1):
            self.nodes[i].connect(self.myelins[i], 1, 0)
            self.myelins[i+1].connect(self.nodes[i], 1, 0)

        self.nodes[node_num-1].connect(self.myelins[node_num-1], 1, 0)
        self.myelins[0].connect(self.ais, 1, 0)
        # if connect == 1:
        #     self.collateral.connect(self.nodes[connection_node], 0.5, 0)

        # Initialise parameters
        Ena = 60
        Ek = -90
        v_init = -68
        rm = 30000
        celsius = 37
        ra = 150
        c_m = 0.8

        #Storing vectors 
        self.V_c_vectors = {} # Store collateral voltage vectors
        self.V_a_vectors = {}  # Store axon voltage vectors
        self.V_ais_vectors = {} # Store AIS voltage vectors
        self.Ext_potential = {} # Store collateral extracellular potential vectors
        self.Firing_Counter_Collateral = [] # Store collateral firing count vectors
        self.Firing_Times_Collateral = []
        self.Firing_Count_Collateral = []
        self.Firing_Counter_Axon = []
        self.Firing_Times_Axon = []
        self.Firing_Count_Axon = []
        
        # Insert mechanisms and define geometries
        for n in self.nodes:
            n.L = 2
            n.diam = N_diam
            n.Ra = ra
            n.insert('cortical_axon_i_kd')
            n.insert('cortical_axon_i_kv')
            n.insert("cortical_axon_i_na")
            n.insert('cortical_axon_i_leak')
            n.insert(h.extracellular)
            #n.insert(h.APCount)
            n.g_Na_cortical_axon_i_na = 0.28
            n.g_Kd_cortical_axon_i_kd = 0.0072
            n.g_Kv_cortical_axon_i_kv = 0.0005
            n.g_l_cortical_axon_i_leak = 0.02
            n.ek = Ek
            n.ena = Ena
            n.cm = c_m
            n.nseg = 5

        for m in self.myelins:
            m.L = 500
            m.diam = M_diam
            m.Ra = ra
            m.cm = 0.04
            m.nseg = 11
            m.insert('cortical_axon_i_na')
            m.insert('cortical_axon_i_leak')
            m.g_Na_cortical_axon_i_na = 0.001
            m.g_l_cortical_axon_i_leak = 0
            m.ena = Ena
        self.myelins[0].L = 80

        self.ais.L = 20
        self.ais.nseg = 5
        self.ais.diam = N_diam
        self.ais.insert('cortical_axon_i_na')
        self.ais.insert('cortical_axon_i_kv')
        self.ais.insert('cortical_axon_i_kd')
        self.ais.insert('cortical_axon_i_leak')
        self.ais.insert(h.extracellular)
        self.ais.g_Kv_cortical_axon_i_kv = 0.002
        self.ais.g_Na_cortical_axon_i_na = 0.4
        self.ais.g_Kd_cortical_axon_i_kd = 0.015
        self.ais.g_l_cortical_axon_i_leak = 0.000033
        self.ais.Ra = ra
        self.ais.cm = c_m
        self.ais.ek = Ek
        self.ais.ena = Ena


    @staticmethod
    def Euclidean(x1, x2, y1, y2):
        return m.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    def Coordinates(self, Electrode_X_d, Electrode_Y_d):
    
        # Axon Coords
        y_coords_A = []
        for n in range(self.node_num):
            if n == 0:
                y_coords_A.append(20 +80+ 1)  # AIS + Myelin[0] + midpoint of node 0
            else:
                y_coords_A.append(20 +80 +  500* n + n * 2 + 1)  # AIS + initial myelin + num_myelin x n + nodes x node_length + 1 for centre of that node

        Node_radius = (self.nodes[0].diam) / 2
        X_A = Node_radius  # x coordinate of axon is radius of node
        r_A = []  # euclidean distance for the axon

        for n in range(self.node_num):
            r_A.append(Cortical_Neuron.Euclidean(X_A, Electrode_X_d, y_coords_A[n], Electrode_Y_d))

        # AIS coords
        Nseg_AIS  = self.ais.nseg
        AIS_L = self.ais.L
        X_AIS = Node_radius
        y_coords_AIS = []
        for seg in range(Nseg_AIS):
            y_coords_AIS.append((seg / Nseg_AIS) * AIS_L)

        r_AIS = []
        for seg in range(Nseg_AIS):
            r_AIS.append(Cortical_Neuron.Euclidean(X_AIS, Electrode_X_d, y_coords_AIS[seg], Electrode_Y_d))
            
        # conversion factor
        for i in range(len(r_A)):
            r_A[i] *= 10**-6

        # for i in range(len(r_C)):
        #     r_C[i] *= 10**-6

        for i in range(len(r_AIS)):
            r_AIS[i] *= 10**-6
            
        return r_A, r_AIS, y_coords_A, Node_radius, Electrode_X_d, Electrode_Y_d

    def Plot_Geometry(self, Electrode_X_d=1500, Electrode_Y_d=521):
        [r_A, r_AIS, y_coords_A, Node_radius, Electrode_X_d, Electrode_Y_d] = self.Coordinates(Electrode_X_d, Electrode_Y_d)
        # Plot coordinates of Electrode, collateral and axon with respect to y and x axis:
        # y_coords_C = []
        x_coords_A = []
        # for i in range(len(x_coords_C)):
        #     y_coords_C.append(Collateral_YPosition)

        for i in range(len(y_coords_A)):
            x_coords_A.append(Node_radius)

        stim_x = Electrode_X_d
        stim_y = Electrode_Y_d
        #plt.plot(x_coords_C, y_coords_C, '--', color='red', label='Collateral Position')
        plt.plot(x_coords_A, y_coords_A, color='blue', label='Axon Position')
        plt.plot(stim_x, stim_y, '*', color='black', label='Electrode Position')
        plt.title('Relative Position of Axon and Electrode')
        plt.xlabel('X (um)')
        plt.ylabel('Y (um)')
        plt.legend()
        plt.grid(True)
        plt.show()

    def Extracellular(self, amp_A, node_num, delay, pulse_dur, total_dur, Electrode_X_d=1500, Electrode_Y_d=521):
        [r_A, r_AIS, y_coords_A, Node_radius, Electrode_X_d, Electrode_Y_d] = self.Coordinates(Electrode_X_d, Electrode_Y_d)
        pi = 3.141
        sigma = 0.2  # S/m
        Phi_A = []
        for i in range(len(r_A)):
            Phi_A.append((amp_A / (r_A[i] * sigma * pi * 4))*10**3)


        Phi_AIS = []
        for i in range(len(r_AIS)):
            Phi_AIS.append((amp_A / (r_AIS[i] * sigma * pi * 4))*10**3)


        # Time vector
        t0 = 0
        t1 = delay
        t2 =delay
        t3 = delay + pulse_dur
        t4 = delay + pulse_dur
        t5 = total_dur  # 15ms
        stimtvec_list_A = [t0, t1, t2, t3, t4, t5]
        stimtvec_A = h.Vector(stimtvec_list_A)

    

        stimtvec_list_AIS = [t0, t1, t2, t3, t4, t5]
        stimtvec_AIS = h.Vector(stimtvec_list_AIS)


        #Axon
        ampvec_allnodes_A = []
        for n in range(node_num):
            stimampvec_list_A = [0, 0, Phi_A[n], Phi_A[n], 0, 0]
            ampvec_allnodes_A.append(h.Vector(stimampvec_list_A))
            ampvec_allnodes_A[-1].play(self.nodes[n](0.5)._ref_e_extracellular, stimtvec_A, True)
            #print(f"{n}: {stimampvec_list_A}")
        


        #AIS
        Nseg_AIS  = self.ais.nseg
        ampvec_allnodes_AIS = []
        for i in range(Nseg_AIS):
            loc = i / Nseg_AIS
            stimampvec_list_AIS = [0, 0, Phi_AIS[i], Phi_AIS[i], 0, 0]
            ampvec_allnodes_AIS.append(h.Vector(stimampvec_list_AIS))
            ampvec_allnodes_AIS[-1].play(self.ais(loc)._ref_e_extracellular, stimtvec_AIS, True)

        return ampvec_allnodes_A, ampvec_allnodes_AIS, stimtvec_A, stimtvec_AIS

            
    def Record_time(self):
        t = h.Vector().record(h._ref_t)
        return t


    def record_voltage_axon(self, node_index):
        # Records the voltage but does not return it
        v_vec2 = h.Vector() 
        v_vec2.record(self.nodes[node_index](0.5)._ref_v)
        self.V_a_vectors[node_index]=v_vec2

    def record_voltage_AIS(self,seg_index):
        # Records the voltage but does not return it
        loc = seg_index/self.ais.nseg
        v_vec3 = h.Vector() 
        v_vec3.record(self.ais(loc)._ref_v)
        self.V_ais_vectors[seg_index]=v_vec3

    # def get_voltage_collateral(self, seg_index):
    #     # Retrieves recorded voltage data for collateral
    #     return np.array(self.V_c_vectors[seg_index])
        
    def get_voltage_axon(self,node_index):
        # Retrieves recorded voltage data for axon
        return np.array(self.V_a_vectors[node_index])

    def get_voltage_ais(self, seg_index):
        # Retrieves recorded voltage data for collateral
        return np.array(self.V_ais_vectors[seg_index])

    # def get_ex_collateral(self, seg_index):
    #     # Retrieves recorded voltage data for collateral
    #     return np.array(self.Ext_potential[seg_index])

    
    
    def find_spikes(self, voltage_trace, times, height_threshold, artifact_start, artifact_end):    
        artifact_start_idx = (np.abs(times - artifact_start)).argmin()
        artifact_end_idx = (np.abs(times - artifact_end)).argmin()
        
        # Use find_peaks to detect potential spikes based on the height_threshold and other parameters
        initial_spike_indices, _ = find_peaks(voltage_trace, height=height_threshold)
        
        # Filter detected spikes to ensure they are not within the artifact timeframe
        validated_spike_indices = [i for i in initial_spike_indices if not (artifact_start_idx <= i <= artifact_end_idx) and voltage_trace[i] <= 70]  # ensure spike isn't stimulus artifact <70mV
        
        # Convert indices to times using a list comprehension
        spike_times = [times[i] for i in validated_spike_indices] if validated_spike_indices else [np.nan]
        spike_count = len(validated_spike_indices)
        #print(f"Spike count: {spike_count}, Spike times: {spike_times}")
        return spike_times, spike_count    


In [4]:
def check_firing(spike_data):
    firing_counts = {'Axon': 0, 'AIS': 0}  # Include AIS
    for data in spike_data:
        if data[3] > 0:  # If spike_count > 0
            firing_counts[data[0]] += 1
    # Consider AIS firing in the overall firing decision
    return firing_counts['Axon'] > 2 or firing_counts['AIS'] > 0


def simulate_and_check_firing(current_amplitude, electrode_x_pos, electrode_y_pos,  node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam):  
    h.tstop = total_dur
    h.celsius = 37
    h.finitialize(-68)
    C = Cortical_Neuron(node_num, M_diam, N_diam)
    Electrode_X_d = electrode_x_pos
    Electrode_Y_d = electrode_y_pos
    [ampvec_allnodes_A, ampvec_allnodes_AIS, stimtvec_A, stimtvec_AIS] = C.Extracellular(current_amplitude, node_num, delay, pulse_dur, total_dur, Electrode_X_d, Electrode_Y_d)
    TH = 0
   
    #t = C.Record_time()
    t = np.linspace(0, 100, 4001)

    # Recording voltage and running simulation
    for i in range(node_num):
        C.record_voltage_axon(i)
    # for j in range(C.collateral.nseg):
    #     C.record_voltage_collateral(j)
    for k in range(C.ais.nseg):
        C.record_voltage_AIS(k)

    h.run()  # Run the simulation

    # Gathering spike data and checking for firing
    spike_data = []

    for n in range(node_num):
        va = C.get_voltage_axon(n)
        spike_times, spike_count = C.find_spikes(va, t, TH, artifact_start, artifact_end)
        if spike_times and not np.isnan(spike_times[0]):  # Ensure there's at least one valid spike time
            first_spike_voltage = va[(np.abs(t - spike_times[0])).argmin()]
        else:
            first_spike_voltage = None
        spike_data.append(('Axon', n, spike_times, spike_count, first_spike_voltage))


    for seg in range(C.ais.nseg):
        vais = C.get_voltage_ais(seg)
        spike_times, spike_count = C.find_spikes(vais, t, TH, artifact_start, artifact_end)
        if spike_times and not np.isnan(spike_times[0]):  # Ensure there's at least one valid spike time
            first_spike_voltage = vais[(np.abs(t - spike_times[0])).argmin()]
        else:
            first_spike_voltage = None
        spike_data.append(('AIS', seg, spike_times, spike_count, first_spike_voltage))

    del C
    if check_firing(spike_data):
        # Once firing is confirmed, we can returm this and the current amplitude
        return True, current_amplitude
    else:
        return False, current_amplitude

def find_threshold_binary_search(electrode_x_pos, electrode_y_pos, min_amp, max_amp, tolerance, node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam):
    last_firing_amp = None  # Initialize to track the last amplitude that caused firing
    firing_observed = False  # Initialize firing_observed flag
    while abs(max_amp - min_amp) > tolerance:
        mid_amp = (max_amp + min_amp) / 2
        firing, _ = simulate_and_check_firing(mid_amp, electrode_x_pos, electrode_y_pos, node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam)
        if firing:
            last_firing_amp = mid_amp  # Update last firing amplitude
            firing_observed = True  # Update firing observed flag
            max_amp = mid_amp  # Firing detected, lower the upper bound
        else:
            min_amp = mid_amp  # No firing, raise the lower bound

    # Ensure a firing threshold is returned if observed, else return the final midpoint
    threshold_value = last_firing_amp if last_firing_amp is not None else (max_amp + min_amp) / 2
    return threshold_value, firing_observed



def verify_threshold(current_threshold, electrode_x_pos, electrode_y_pos, node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam):
    # Initial check at the current threshold
    firing_at_threshold, verfied_threshold = simulate_and_check_firing(current_threshold, electrode_x_pos, electrode_y_pos, node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam)
    if firing_at_threshold:
        print(f"Neuron fires at the current threshold {current_threshold} A. ")
        Verify = True
    else:
        Verify = False
        print(f"Neuron at {current_threshold} did not fire from verify function")
       
    return Verify,verfied_threshold




In [5]:
def inspect_voltage_trace(voltage_trace, segment_type, index, t, TH, artifact_start, artifact_end):
    # Find spike times outside the artifact period and above threshold
    spike_times_indices = np.where((voltage_trace > TH) & ((t > artifact_end) | (t < artifact_start)))[0]
    # Convert indices to actual times
    spike_times = t[spike_times_indices]
    # Pair each spike time with its location
    spike_info = [(time, (segment_type, index)) for time in spike_times]
    return spike_info


def identify_earliest_spike_and_location(current_amplitude, electrode_x_pos, electrode_y_pos, node_num, delay, pulse_dur, total_dur, artifact_start, artifact_end,  TH, M_diam, N_diam):
    # Set up the simulation parameters
    h.tstop = total_dur
    h.celsius = 37
    h.finitialize(-68)
    
    # Create the neuron model
    C = Cortical_Neuron(node_num, M_diam, N_diam)

    # Initialize variables to record the earliest spike time and location
    earliest_spike_time = float('inf')
    earliest_spike_location = None
    
    # Set the position of the electrode
    Electrode_X_d = electrode_x_pos
    Electrode_Y_d = electrode_y_pos
    
    # Apply extracellular stimulation
    [ampvec_allnodes_A, ampvec_allnodes_AIS, stimtvec_A, stimtvec_AIS] = C.Extracellular(current_amplitude, node_num, delay, pulse_dur, total_dur, Electrode_X_d, Electrode_Y_d)
    
    # Record the time before running the simulation
    #t = C.Record_time()
    t = np.linspace(0, 100, 4001)
    

    # Recording voltage and running simulation
    for i in range(node_num):
        C.record_voltage_axon(i)
        
    for k in range(C.ais.nseg):
        C.record_voltage_AIS(k)
    
    # Run the simulation
    h.run()

    all_spikes = []  # To collect spikes from all segments

    # Axon segments
    for i in range(node_num):
        voltage_trace = C.get_voltage_axon(i)
        spike_times_indices_A = np.where((voltage_trace > TH) & ((t > artifact_end) | (t < artifact_start)))[0]
        
        for idx in spike_times_indices_A:
            spike_time = t[idx]
            all_spikes.append((spike_time, ('Axon', i)))
   

    # AIS segments
    for k in range(C.ais.nseg):
        voltage_trace = C.get_voltage_ais(k)
        spike_times_indices_AIS = np.where((voltage_trace > TH) & ((t > artifact_end) | (t < artifact_start)))[0]

        for idx in spike_times_indices_AIS: 
            spike_time = t[idx]
            all_spikes.append((spike_time, ('AIS', k)))
   
    # Find the earliest spike time and location
    if all_spikes:
        earliest_spike = min(all_spikes, key=lambda x: x[0])  # Minimize by spike time
        return earliest_spike[0], earliest_spike[1]  # (time, location)
    else:
        return None, None  # No spikes detected

In [6]:
# Constants and initial setup for the test
node_num = 66
delay = 80
pulse_dur = 60 * (10**-3)
artifact_start = delay
artifact_end = delay + pulse_dur
total_dur = 100
electrode_y_positions = [16000, 20000, 24000]
x_positions= [1000, 1250, 1500] #validation points
Node_Diameters= [1.2,1.59,1.98,2.36,2.75,3.14,3.53,3.92,4.30,4.69,5.08,5.47,5.86,6.24,6.63,7.02,7.41,7.80,8.18,8.57]
Myelin_Diameters= [1.4,1.85,2.31,2.76,3.21,3.66,4.12,4.57,5.02,5.47,5.93,6.38,6.83,7.28,7.74,8.19,8.64,9.09,9.55,10.00]  
polarity = -1  # +1 for Anodal, -1 for Cathodal
TH = 0
min_current = 0.0001 * polarity
max_current = 0.05 * polarity
tolerance = 0.000001  # Fine tolerance for more precise threshold detection
run_no = 1
polarity_label = "Anodal" if polarity == 1 else "Cathodal"
results = []
Unverified_Positions = []

for idx, M_diam in enumerate(Myelin_Diameters):
    N_diam = Node_Diameters[idx]
    
    for y_pos in electrode_y_positions:
        for x_pos in x_positions:
            
            # Step 1: Find and verify the threshold
            initial_threshold, firing_observed = find_threshold_binary_search(
                x_pos, y_pos, min_current, max_current, tolerance, node_num,
                delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam
            )
            if firing_observed:
                verified, verified_threshold = verify_threshold(
                    initial_threshold, x_pos, y_pos, node_num,
                    delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam
                )
                
                if verified:
                    # Step 2: Identify the earliest spike time and location for the verified threshold
                    earliest_spike_time, earliest_spike_location = identify_earliest_spike_and_location(
                        verified_threshold, x_pos, y_pos, node_num,
                        delay, pulse_dur, total_dur, artifact_start, artifact_end, TH, M_diam, N_diam
                    )
                    
                    # Append results including the earliest spike information
                    results.append({
                        'Run No': run_no,
                        'Electrode Y Position': y_pos,
                        'Electrode X Position': x_pos,
                        'Current Polarity': polarity_label,
                        'Threshold (A)': verified_threshold,
                        'Activation Location': earliest_spike_location,
                        'First Spike Time': earliest_spike_time,
                        'Myelin Diameter': M_diam,
                        'Node Diameter': N_diam,
                        
                    })
                else:
                    print(f"Current threshold for X = {x_pos} , Y = {y_pos}, could not be verified, needs further checks")
                    Unverified_Positions.append((x_pos, y_pos))
            else:
                print(f"No firing achieved for X={x_pos}, Y={y_pos} at any tested amplitude up to {max_current}.")
                results.append({
                    'Run No': run_no,
                    'Electrode Y Position': y_pos,
                    'Electrode X Position': x_pos,
                    'Current Polarity': polarity_label,
                    'Threshold (A)': 'No firing detected',
                    'Activation Location': 'None',
                    'First Spike Time': None,
                    'Myelin Diameter': M_diam,
                    'Node Diameter': N_diam,
                    
                })
            
            run_no += 1  # Increment run number for the next iteration
            print(f"Run Number: {run_no}")

# Convert results to a DataFrame and optionally save
results_df = pd.DataFrame(results)
# results_df.to_csv("simulation_results.csv", index=False)


Neuron fires at the current threshold -0.005067462158203124 A. 
Run Number: 2
Neuron fires at the current threshold -0.005072792053222657 A. 
Run Number: 3
Neuron fires at the current threshold -0.0050796447753906245 A. 
Run Number: 4
Neuron fires at the current threshold -0.00633140869140625 A. 
Run Number: 5
Neuron fires at the current threshold -0.0063359771728515625 A. 
Run Number: 6
Neuron fires at the current threshold -0.006341307067871093 A. 
Run Number: 7
Neuron fires at the current threshold -0.007595355224609376 A. 
Run Number: 8
Neuron fires at the current threshold -0.00759916229248047 A. 
Run Number: 9
Neuron fires at the current threshold -0.007603730773925782 A. 
Run Number: 10
Neuron fires at the current threshold -0.004721780395507812 A. 
Run Number: 11
Neuron fires at the current threshold -0.004727110290527343 A. 
Run Number: 12
Neuron fires at the current threshold -0.004733201599121094 A. 
Run Number: 13
Neuron fires at the current threshold -0.005898925781250001 

In [7]:
# ### Display Section
# pd.set_option('display.precision', 19)
# # Display the summary for each specified y position
# summary_columns = ['Electrode Y Position', 'Electrode X Position', 'Activtion Location', 'Threshold (A)', 'First Spike Time', 'Myelin Diameter', 'Node Diameter']
# summary_df = results_df[summary_columns]  # Assuming all rows have valid data

# # Iterate through each specified y position for display
# for specific_y_pos in electrode_y_positions:
#     print(f"Data for Electrode Y Position {specific_y_pos}:")
#     display(summary_df[summary_df['Electrode Y Position'] == specific_y_pos])
#     print("\n")  # Add a newline for better separation between each y position's data


In [8]:
#results_df.to_csv("Axon_Diameter_results_Cathodal_lengthened2.csv", index=False)
results_df.to_csv("Axon_Diameter_cathodal_no_ratio_results_further.csv", index=False)
